In [1]:
include("src/cpa.jl")

run_method (generic function with 1 method)

# Full run function
- all methods in comp for tfim 127 nq

In [2]:
global_logger(UnbufferedLogger(stdout,SubInfo))

Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(31) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

In [3]:
function full_run_all_methods(ansatz::trotter_ansatz_tfim,
    angle_definition::Float64,
    noise_kind::String;
    min_abs_coeff::Float64=0.0,
    min_abs_coeff_noisy::Float64=0.0,
    max_weight=Inf,
    training_set=nothing,
    observable=nothing,
    num_samples::Int=10,
    depol_strength::Float64=0.01,
    dephase_strength::Float64=0.01,
    depol_strength_double::Float64=0.0033,
    dephase_strength_double::Float64=0.0033,
    min_abs_coeff_target::Float64=0.0,
    noise_levels::Vector{Float64}=[1.0,1.5,2.0],
    lambda::Float64=0.0,
    use_target::Bool=true,
    real_qc_noisy_data=nothing
)
    @logmsg SubInfo "→ Starting full_run_all_methods (noise_kind=$noise_kind, σ=$angle_definition)"

    # determine observable name
    obs_62 = PauliSum(ansatz.nqubits); add!(obs_62, :Z, 62)
    if observable === nothing || observable == obs_interaction(ansatz)
    observable, obs_string = obs_interaction(ansatz), "ZZ"
    elseif observable == obs_magnetization(ansatz)
    obs_string = "Z"
    elseif observable == obs_62
    obs_string = "Z_62"
    else
    obs_string = "Unknown"
    end
    @logmsg SubInfo "→ Observable: $obs_string"

    # generate training set if needed
    if training_set === nothing
    @logmsg SubInfo "→ Generating training set (n=$num_samples)…"
    training_set = training_set_generation_strict_perturbation(ansatz, angle_definition; num_samples=num_samples)
    end
    @logmsg SubInfo "→ Training set size: $(length(training_set))"

    # timestamp start
    time1 = time()

    # compute or inject final‐step target values
    if use_target
    exact_target = trotter_time_evolution(ansatz; observable=observable,
                    noise_kind="noiseless",
                    min_abs_coeff=min_abs_coeff_target,
                    record=false)
    timetmp = time()
    @logmsg SubInfo "→ exact_target done in $(round(timetmp - time1; digits=2)) s"

    noisy_target = trotter_time_evolution(ansatz; observable=observable,
                    noise_kind=noise_kind,
                    min_abs_coeff=min_abs_coeff_target,
                    record=false)
    timetmp = time()
    @logmsg SubInfo "→ noisy_target done in $(round(timetmp - time1; digits=2)) s"
    else
    @logmsg SubInfo "→ Skipping model targets; using real_qc_noisy_data=$(real_qc_noisy_data)"
    exact_target = NaN
    noisy_target = real_qc_noisy_data !== nothing ? real_qc_noisy_data : NaN
    timetmp = time()
    end
    @logmsg SubInfo "→ Targets: exact=$(exact_target), noisy=$(noisy_target)"

    # build training ensembles
    exact_train = training_trotter_time_evolution(ansatz, training_set;
                        observable=observable,
                        noise_kind="noiseless",
                        min_abs_coeff=min_abs_coeff,
                        max_weight=max_weight,
                        record=false)
    timetmp1 = time()
    @logmsg SubInfo "→ exact_train done in $(round(timetmp1 - timetmp; digits=2)) s"

    noisy_train = training_trotter_time_evolution(ansatz, training_set;
                        observable=observable,
                        noise_kind=noise_kind,
                        min_abs_coeff=min_abs_coeff_noisy,
                        max_weight=max_weight,
                        depol_strength=depol_strength,
                        dephase_strength=dephase_strength,
                        depol_strength_double=depol_strength_double,
                        dephase_strength_double=dephase_strength_double,
                        record=false)
    timetmp2 = time()
    @logmsg SubInfo "→ noisy_train done in $(round(timetmp2 - timetmp1; digits=2)) s"

    # --- ZNE ---
    zne_levels = zne_time_evolution(ansatz; observable=observable,
            noise_kind=noise_kind,
            min_abs_coeff=min_abs_coeff,
            max_weight=max_weight,
            noise_levels=noise_levels,
            depol_strength=depol_strength,
            dephase_strength=dephase_strength,
            depol_strength_double=depol_strength_double,
            dephase_strength_double=dephase_strength_double,
            record=false)
    result_zne = zne(zne_levels;
    noise_levels=noise_levels,
    fit_type="linear",
    exact_target_exp_value = use_target ? exact_target : nothing,
    use_target=use_target)
    timetmp3 = time()
    @logmsg SubInfo "→ ZNE done in $(round(timetmp3 - timetmp2; digits=2)) s"
    if use_target
    zne_corr, zne_err_after, zne_err_before = result_zne
    else
    zne_corr = result_zne; zne_err_before = NaN; zne_err_after = NaN
    end
    open("trotter_ZNE_$(noise_kind).log","a") do log
    str = format(
    "{:>10s} {:>3n}{:>6.2e}{:>10.2e}{:>10.2e} {:>2s} {:>5n} {:>10.3e} {:>10.3e} {:>10.3e} {:>10.3e} {:>8.2f}\n",
    "ZNE", ansatz.steps,
    ansatz.time, ansatz.J, ansatz.h,
     obs_string,
      ansatz.nqubits,
    exact_target, 
    noisy_target,
    zne_err_before, zne_err_after,
    timetmp3 - time1
    )
    write(log, str)
    end

    # --- CDR ---
    result_cdr = cdr(noisy_train, exact_train, noisy_target;
    exact_target_exp_value = use_target ? exact_target : nothing,
    use_target=use_target)
    timetmp4 = time()
    @logmsg SubInfo "→ CDR done in $(round(timetmp4 - timetmp3; digits=2)) s"
    if use_target
    cdr_corr, cdr_err_after, cdr_err_before = result_cdr
    else
    cdr_corr = result_cdr; cdr_err_before = NaN; cdr_err_after = NaN
    end
    open("trotter_CDR_$(noise_kind).log","a") do log
    str = format(
    "{:>10s} {:>3n}{:>6.2e}{:>10.2e}{:>10.2e} {:>2s} {:>5n} {:>10.3e} {:>10.3e} {:>10.3e} {:>10.3e} {:>8.2f}\n",
    "CDR", ansatz.steps,ansatz.time, ansatz.J, ansatz.h, obs_string, ansatz.nqubits,
    exact_target, noisy_target,
    cdr_err_before, cdr_err_after,
    timetmp4 - time1
    )
    write(log, str)
    end

    # --- vnCDR ---
    noisy_train_multi = vnCDR_training_trotter_time_evolution(ansatz, training_set;
                                    observable=observable,
                                    noise_kind=noise_kind,
                                    min_abs_coeff=min_abs_coeff,
                                    max_weight=max_weight,
                                    noise_levels=noise_levels,
                                    depol_strength=depol_strength,
                                    dephase_strength=dephase_strength,
                                    depol_strength_double=depol_strength_double,
                                    dephase_strength_double=dephase_strength_double,
                                    record=false)
    result_vn = vnCDR(noisy_train_multi, exact_train, zne_levels;
    exact_target_exp_value = use_target ? exact_target : nothing,
    use_target=use_target, lambda=lambda)
    timetmp5 = time()
    @logmsg SubInfo "→ vnCDR done in $(round(timetmp5 - timetmp4; digits=2)) s"
    if use_target
    vn_corr, vn_err_after, vn_err_before = result_vn
    else
    vn_corr = result_vn; vn_err_before = NaN; vn_err_after = NaN
    end
    open("trotter_vnCDR_$(noise_kind).log","a") do log
    str = format(
    "{:>10s} {:>3n}{:>6.2e}{:>10.2e}{:>10.2e} {:>2s} {:>5n} {:>10.3e} {:>10.3e} {:>10.3e} {:>10.3e} {:>8.2f}\n",
    "vnCDR", ansatz.steps,ansatz.time, ansatz.J, ansatz.h, obs_string, ansatz.nqubits,
    exact_target, noisy_target,
    vn_err_before, vn_err_after,
    timetmp5 - time1
    )
    write(log, str)
    end

    @logmsg SubInfo "→ full_run_all_methods complete."
    return (exact_target, noisy_target,
    zne_corr, cdr_corr, vn_corr,
    zne_err_before, cdr_err_before, vn_err_before,
    zne_err_after,  cdr_err_after,  vn_err_after)
end

full_run_all_methods (generic function with 1 method)

In [ ]:
function IBM_utility_exp_4b_all()
    global_logger(UnbufferedLogger(stdout, SubInfo))
    
    nq, nl, T = 8, 2, nl/20
    topology = bricklayertopology(nq)
    IBM_angles = [0.3,1.0,0.7,0.0,0.2,0.8,0.5,0.1,0.4,1.5707,0.6]
    h_values = IBM_angles .* nl/(2*T)
    println("h_values = ", h_values)
    
    IBM_unmitigated_vals =  [ 0.4188991191900761,
    0.004107759335343423,
    0.11944580478416555,
    0.49038646460776864,
    0.4552471452020139,
    0.055064655494323766,
    0.3061535376123831,
    0.4889782663914668,
    0.3622122171682965,
   -0.001980699802309258,
    0.20175539633925924]
    
    noise_kind="gate_kickedising"; sigma_star=π/20
    min_abs_coeff, min_abs_coeff_noisy, max_weight = 1e-5,1e-5,20
    depol_strength,dephase_strength = 0.01,0.01
    depol_strength_double,dephase_strength_double = 0.0033,0.0033
    noise_levels=[1.0,1.5,2.0]; lambda=0.0; use_target=false; cdr_method="end"
    
    observable = PauliSum(nq); add!(observable,:Z,4)
    collect_exact = Float64[]; collect_noisy = Float64[]
    collect_zne = Float64[]; collect_cdr = Float64[]; collect_vncd = Float64[]
    
    for (i,h) in enumerate(h_values)
    println("→ h = $h")
    trotter = trotter_kickedising_setup(nq, nl, T, h; topology=topology)
    training_set = training_set_generation_strict_perturbation(trotter; sample_function="small", num_samples=10)
    
    exact, noisy,
    zne_corr, cdr_corr, vn_corr,
    _, _, _,
    _, _, _ = full_run_all_methods(
    trotter, sigma_star, noise_kind;
    min_abs_coeff=min_abs_coeff, max_weight=max_weight,
    min_abs_coeff_noisy=min_abs_coeff_noisy,
    training_set=training_set, observable=observable,
    num_samples=10,
    depol_strength=depol_strength, dephase_strength=dephase_strength,
    depol_strength_double=depol_strength_double, dephase_strength_double=dephase_strength_double,
    noise_levels=noise_levels, lambda=lambda,
    use_target=use_target,
    real_qc_noisy_data=IBM_unmitigated_vals[i]
    )
    
    push!(collect_exact, exact)
    push!(collect_noisy, noisy)
    push!(collect_zne, zne_corr)
    push!(collect_cdr, cdr_corr)
    push!(collect_vncd, vn_corr)
    end
    
     # log file for this utility run, stamped with current datetime
     run_ts = Dates.format(Dates.now(), "yyyyMMdd_HHMMSS")
     logfname = "tfim_utility_nq=$(nq)_angle_def=$(sigma_star)_$(run_ts).log"
     println("→ Logging summary to $logfname")
     
    # write summary table to log
    open(logfname, "a") do io
        # header
        println(io, "idx,Exact_targets,Noisy_targets,ZNE_outputs,CDR_outputs,vnCDR_outputs")
        # rows
        for i in 1:length(collect_exact)
            println(io, join((
                i,
                collect_exact[i],
                collect_noisy[i],
                collect_zne[i],
                collect_cdr[i],
                collect_vncd[i]
            ), ","))
        end
    end
    # log file for this utility run, stamped with current datetime
    run_ts = Dates.format(Dates.now(), "MMdd_HHMMSS")
    logfname = "tfim_utility_nq=$(nq)_angle_def=$(round(sigma_star;digits=3))_$(run_ts).log"
    @logmsg SubInfo "→ Logging summary to $logfname"
     # write summary table to log
     open(logfname, "a") do log
        # header
        write(log, "idx,Exact_targets,Noisy_targets,ZNE_outputs,CDR_outputs,vnCDR_outputs\n")
        # rows
        for i in 1:length(collect_exact)
            write(log, join((i,
                              collect_exact[i],
                              collect_noisy[i],
                              collect_zne[i],
                              collect_cdr[i],
                              collect_vncd[i]), ',') * "\n")
        end
    end

    println("Exact targets:   ", collect_exact)
    println("Noisy targets:   ", collect_noisy)
    println("ZNE outputs:     ", collect_zne)
    println("CDR outputs:     ", collect_cdr)
    println("vnCDR outputs:   ", collect_vncd)
    
    end
    
    IBM_utility_exp_4b_all()
    

h_values = [2.9999999999999996, 10.0, 6.999999999999999, 0.0, 2.0, 8.0, 5.0, 1.0, 4.0, 15.706999999999999, 5.999999999999999]
→ h = 2.9999999999999996
[SubInfo] → Starting full_run_all_methods (noise_kind=gate_kickedising, σ=0.15707963267948966)
[SubInfo] → Observable: Unknown
[SubInfo] → Training set size: 10
[SubInfo] → Skipping model targets; using real_qc_noisy_data=0.4188991191900761
[SubInfo] → Targets: exact=NaN, noisy=0.4188991191900761
[SubInfo] → exact_train done in 0.0 s
[SubInfo] → noisy_train done in 0.0 s
[SubInfo] → ZNE done in 0.0 s
[SubInfo] → CDR done in 0.0 s
Noisy expval at noise level 1.0: [0.9594489523615278, 0.9599590603937079, 0.9596910449717098, 0.960591402531914, 0.9602655155685925, 0.9596451474585619, 0.9598396622711652, 0.9591533201548925, 0.9605942943846643, 0.959198507626822]
Noisy expval at noise level 1.5: [0.9397901253598993, 0.9404553126312363, 0.9401025694818175, 0.9413298649721458, 0.94087002631631, 0.9400429228629681, 0.940297177550917, 0.9394150133